
# Linear Relaxation of Softsign

In [11]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
model = Sequential()
model.add(Activation('softsign', input_dim=1))

In [4]:
import sys
sys.path.append('../..')
import decomon
from decomon.models import convert, clone
from decomon import get_upper_box, get_lower_box, get_range_box

In [5]:
decomon_model = convert(model)

In [6]:
def softsign(x):
    return x/(np.abs(x)+1.)


In [13]:
nn_layers = [Dense(10, activation='relu', input_dim=1), Dense(10, activation='relu'),\
            Dense(1)]

nn_model = Sequential(nn_layers)

In [14]:
nn_model.compile('adam', 'mse')

In [21]:
x_ =np.linspace(-10, 10, 1000)
nn_model.fit(x_, model.predict(x_), epochs=10)

Epoch 1/10
32/32 [==============================] - 0s 785us/step - loss: 2.1177e-04
Epoch 2/10
32/32 [==============================] - 0s 676us/step - loss: 1.9849e-04
Epoch 3/10
32/32 [==============================] - 0s 720us/step - loss: 1.9057e-04
Epoch 4/10
32/32 [==============================] - 0s 754us/step - loss: 1.7154e-04
Epoch 5/10
32/32 [==============================] - 0s 790us/step - loss: 1.6040e-04
Epoch 6/10
32/32 [==============================] - 0s 580us/step - loss: 1.5209e-04
Epoch 7/10
32/32 [==============================] - 0s 596us/step - loss: 1.4004e-04
Epoch 8/10
32/32 [==============================] - 0s 703us/step - loss: 1.3499e-04
Epoch 9/10
32/32 [==============================] - 0s 532us/step - loss: 1.2533e-04
Epoch 10/10
32/32 [==============================] - 0s 618us/step - loss: 1.2094e-04


In [22]:
def linear_hull_upper_decomon(lower, upper):
    x_min = lower*np.ones((1,1))
    x_max = upper*np.ones((1,1))
    box = np.concatenate([x_min[:,None], x_max[:,None]], 1)
    #print(box)
    output = decomon_model.predict([x_max, box])
    _, _, upper, _, w_u, b_u, _, _ = output

    return upper.min(), w_u.min(), b_u.min()

def linear_hull_lower_decomon(lower, upper):
    
    x_min = lower*np.ones((1,1))
    x_max = upper*np.ones((1,1))
    box = np.concatenate([x_min[:,None], x_max[:,None]], 1)
    
    output = decomon_model.predict([x_min, box])
    _, _, _, lower, _, _, w_l, b_l = output
    return lower.min(), w_l.min(), b_l.min()

In [23]:
def func(x_min, x_max):
    n_step=100
    if x_max <x_min:
        x_max_ = x_max + 0
        x_max = x_min
        x_min = x_max_
    x = np.linspace(x_min, x_max, n_step)
    y = np.linspace(-10, 10, 1000)
    
    _,w_u, b_u = linear_hull_upper_decomon(x_min,x_max)
    _, w_l, b_l = linear_hull_lower_decomon(x_min, x_max)
    
    s_z = nn_model.predict(x)
    s_x = softsign(x)
    s_y = softsign(y)
    z_x = w_u*x + b_u
    q_x = w_l*x + b_l

    plt.ylim(-1.3, 1.3)
    
    plt.plot(x, s_x, c='blue')
    plt.plot(x, z_x, c='orange')
    plt.plot(x, q_x, c='green' )
    plt.plot(x, s_z, c='pink')

    plt.plot(y, s_y, '--', c='k')
    plt.title('Linear Relaxation of Softsign')

In [24]:
interact(func,
        x_min = widgets.FloatSlider(value=-10,
                                               min=-10,
                                               max=10.,
                                               step=0.01, continuous_update=False),
        x_max = widgets.FloatSlider(value=10,
                                               min=-10,
                                               max=10,
                                               step=0.01, continuous_update=False),
        )

interactive(children=(FloatSlider(value=-10.0, continuous_update=False, description='x_min', max=10.0, min=-10…

<function __main__.func(x_min, x_max)>